In [18]:
import pandas as pd
import glob
import os
import torch

from collections import defaultdict

In [16]:
files = glob.glob('../data/preprocessed/*.pt')
len(files)

30685

In [35]:
from tqdm import tqdm

meta = defaultdict(list)
for f in tqdm(files):
    pid = os.path.basename(f).split('.')[0]
    data = torch.load(f'../data/preprocessed/{pid}.pt')
    
    if len(data['aa_idx']) != len(data['node_angle_feat']):
        continue
        
    if len(data['aa_idx']) != len(data['node_dir_feat']):
        continue
        
    meta['id'].append(pid)
    meta['num_residues'].append( len(data['aa_idx']) )
    
meta = pd.DataFrame(meta)

100%|██████████| 30685/30685 [00:27<00:00, 1112.66it/s]


In [38]:
meta[meta.num_residues >= 31].to_csv('../data/meta.csv', index=False)

In [40]:
meta[meta.num_residues >= 31].shape

(30446, 2)

In [34]:
meta[meta.num_residues >= 30].shape

(30480, 2)

### GVP-CATH

In [45]:
files = glob.glob('../data/gvp_preprocessed/*.pt')
len(files)

21667

In [46]:
from tqdm import tqdm

meta = defaultdict(list)
for f in tqdm(files):
    pid = os.path.basename(f).split('.')[0]
    data = torch.load(f'../data/gvp_preprocessed/{pid}.pt')
    
    if len(data['aa_idx']) != len(data['node_angle_feat']):
        continue
        
    if len(data['aa_idx']) != len(data['node_dir_feat']):
        continue
        
    meta['id'].append(pid)
    meta['num_residues'].append( len(data['aa_idx']) )
    
meta = pd.DataFrame(meta)

100%|██████████| 21667/21667 [00:28<00:00, 769.20it/s]


In [53]:
import json

with open('../data/gvp-data/chain_set_splits.json') as inFile:
    splits = json.loads(inFile.read())
    
train, val, test = set(splits['train']), set(splits['validation']), set(splits['test'])
train = {id.replace('.', '_') for id in train}
val = {id.replace('.', '_') for id in val}
test = {id.replace('.', '_') for id in test}

In [51]:
len(train), len(val), len(test)

(18024, 608, 1120)

In [54]:
meta['is_train'] = meta['id'].isin(train)
meta['is_val'] = meta['id'].isin(val)
meta['is_test'] = meta['id'].isin(test)

In [60]:
meta[meta.is_train].to_csv('../data/meta.gvp_train.csv', index=False)
meta[meta.is_val].to_csv('../data/meta.gvp_val.csv', index=False)
meta[meta.is_test].to_csv('../data/meta.gvp_test.csv', index=False)